In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers import Attention
import matplotlib.pyplot as plt

In [3]:
# 2. Đọc file csv và gắng index với giá Close
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Analyst/Datasets/AMD Historical Data.csv')
df1=df.reset_index()['Price']

In [4]:
df1.shape

(1611,)

In [5]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [6]:
# 4. Chia train test
train_size = int(0.7 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

In [7]:
train_data.shape

(1127, 1)

In [8]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [9]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [10]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [11]:
X_train.shape

(1026, 100, 1)

In [12]:
# define the input shape
input_data = Input(shape=(None, 1))

# define the LSTM layer
lstm_out = LSTM(32, return_sequences=True)(input_data)

# define the attention layer
attention = Attention()([lstm_out, lstm_out])

# define the output layer
output = Dense(1)(attention)

# create the model
model = Model(inputs=input_data, outputs=output)

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# fit the model to the training data
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
17/17 [==============================] - 5s 104ms/step - loss: 0.0493 - val_loss: 0.0527
Epoch 2/100
17/17 [==============================] - 1s 70ms/step - loss: 0.0377 - val_loss: 0.1152
Epoch 3/100
17/17 [==============================] - 1s 68ms/step - loss: 0.0332 - val_loss: 0.1580
Epoch 4/100
17/17 [==============================] - 1s 73ms/step - loss: 0.0322 - val_loss: 0.1708
Epoch 5/100
17/17 [==============================] - 1s 85ms/step - loss: 0.0318 - val_loss: 0.1933
Epoch 6/100
17/17 [==============================] - 1s 69ms/step - loss: 0.0318 - val_loss: 0.2008
Epoch 7/100
17/17 [==============================] - 2s 105ms/step - loss: 0.0318 - val_loss: 0.2049
Epoch 8/100
17/17 [==============================] - 2s 121ms/step - loss: 0.0320 - val_loss: 0.2006
Epoch 9/100
17/17 [==============================] - 2s 117ms/step - loss: 0.0318 - val_loss: 0.1908
Epoch 10/100
17/17 [==============================] - 1s 74ms/step - loss: 0.0320 - val_loss: 0.

In [14]:
# 10. Dự báo dữ liệu test, val
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)

2/2 [==============================] - 0s 27ms/step


In [15]:
train_predict.shape

(1026, 100, 1)

In [16]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict.reshape(-1, 1))
y_pred=scaler.inverse_transform(y_pred.reshape(-1, 1))
y_pred_val=scaler.inverse_transform(y_pred_val.reshape(-1, 1))

In [17]:
# 12. Đánh giá độ chính xác thuật toán bằng RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

Validation RMSE: 39.63020597839024
Testing RMSE: 40.970385127246495


In [18]:
# 13. Dự báo 30 ngày tiếp theo
x_input=val_data[-time_step:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps= time_step
i=0

# # Reshape the input sequence
# x_input = x_input.reshape((1, n_steps, 1))

while(i<30):
    
    if(len(temp_input)>time_step):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


[[0.20345776]
 [0.20348771]
 [0.20351087]
 [0.20353016]
 [0.20354511]
 [0.203557  ]
 [0.20356712]
 [0.20357494]
 [0.20358126]
 [0.20358549]
 [0.20358707]
 [0.20358987]
 [0.20359595]
 [0.20359933]
 [0.20360203]
 [0.2036042 ]
 [0.20360604]
 [0.20360562]
 [0.203607  ]
 [0.20361371]
 [0.20362078]
 [0.20362456]
 [0.20362595]
 [0.20362832]
 [0.20362933]
 [0.2036292 ]
 [0.20362811]
 [0.20362833]
 [0.20363013]
 [0.20363098]
 [0.20362858]
 [0.20362593]
 [0.20362295]
 [0.20362072]
 [0.20362094]
 [0.20362009]
 [0.20361988]
 [0.20361957]
 [0.20361923]
 [0.20362021]
 [0.20362158]
 [0.20362234]
 [0.20362352]
 [0.20362628]
 [0.20362748]
 [0.20362756]
 [0.20362724]
 [0.20363024]
 [0.20363374]
 [0.20364033]
 [0.20364584]
 [0.20364924]
 [0.20365123]
 [0.2036537 ]
 [0.20365708]
 [0.20365815]
 [0.20365822]
 [0.20365708]
 [0.20365714]
 [0.20365824]
 [0.2036591 ]
 [0.2036589 ]
 [0.20365839]
 [0.20365602]
 [0.2036543 ]
 [0.20365484]
 [0.20365427]
 [0.20365293]
 [0.20365179]
 [0.20365076]
 [0.20364892]
 [0.20

<ipython-input-18-6854ad301121>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_input=np.array(temp_input[1:])


ValueError: ignored

In [ ]:
x_input.shape

In [ ]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Predict','Validate','ValidatePred','Predict30days'])
plt.show()